![Verne](https://www.vernegroup.com/wp-content/uploads/2020/07/LOGO-VERNE-TECHNOLOGY-GROUP-3.png)

# Analizando la cesta de la compra
El análisis de la cesta de la compra, también conocido como análisis de afinidad, es una técnica de modelado basada en la teoría de que, si adquiere un determinado grupo de artículos, es más probable que compre otro. Por ejemplo, alguien que compre mantequilla de cacahuete y pan tiene muchas más probabilidades de querer comprar también mermelada. Sin embargo, no todas las relaciones son tan evidentes. La previsión del comportamiento del consumidor puede aumentar las ventas y ofrecer al comerciante una importante ventaja respecto a sus competidores. En realidad, el análisis de la cesta de la compra no es más que una aplicación de técnicas de análisis de asociación, aunque muchos artículos en línea y tutoriales confundan estos dos conceptos. Para ponerlo en perspectiva respecto a otras técnicas de aprendizaje automático, el análisis de la cesta de la compra es una herramienta de aprendizaje sin supervisión que requiere poca ingeniería de características y una cantidad limitada de limpieza y preparación de datos. En la práctica, las conclusiones que se deducen del análisis de la cesta de la compra se pueden explorar aún más con otras herramientas de IA o ciencia de datos.

A pesar de su capacidad para descubrir patrones ocultos, el análisis de la cesta de la compra es relativamente fácil de explicar y no requiere conocimientos avanzados de cálculo o estadística. Sin embargo, es necesario revisar algunos términos y notaciones convencionales. En primer lugar, se hace referencia a las nociones de causa y efecto como antecedente y consecuente. En el ejemplo que he mencionado antes, la mantequilla de cacahuete y el pan son el antecedente y la mermelada, el consecuente. La notación formal para esta relación sería {Peanut Butter, Bread} -> {Jelly}, lo que indica una conexión entre los dos artículos. Asimismo, hay que tener en cuenta que tanto los antecedentes como los consecuentes pueden constar de varios artículos.

Hay tres medidas matemáticas importantes necesarias para el análisis de la cesta de la compra: soporte, mejora y confianza. El soporte es el número de veces que los antecedentes aparecen juntos en los datos. Para simplificar el ejemplo, imaginemos la relación siguiente: {Peanut Butter} -> {Grape Jelly}. Dados 100 clientes y una transacción por cliente, considere el escenario siguiente:

- 15 clientes compraron mantequilla de cacahuete
- 13 compraron mermelada de uva
- 11 compraron mantequilla de cacahuete y mermelada de uva
El soporte representa el número de veces que los artículos aparecen en una transacción juntos, que, en este ejemplo es 11 de cada 100, o 0,11. En términos estadísticos, hay una probabilidad del 11 por ciento de que cualquier transacción determinada incluya mantequilla de cacahuete y mermelada de uva. La confianza toma el valor del soporte (0,11) y lo divide por la probabilidad de una transacción que tenga solo mermelada de uva, lo que genera un valor de 0,846. Esto significa que casi el 85 por ciento de las veces que se compró mermelada de uva, se compró junto con mermelada de cacahuete. Por último, está la mejora, que toma el valor de la confianza (0,846) y lo divide por la probabilidad de la mantequilla de cacahuete. Esto equivale a 5,64 (redondeado a dos posiciones decimales).


In [36]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [79]:
conn = sqlite3.connect('Tiendas24H.sqlite')
consulta="select v.*,vc.CodTienda from ventasdet v inner join ventascab vc on v.Ticket=vc.Ticket where vc.Ticket in (select distinct Ticket from ventasdet  group by Ticket having count(CodProducto) >1) "

df = pd.read_sql_query(consulta,conn)

df.head()

,index,Ticket,Linea,CodProducto,UnidadMedida,Cantidad,ImporteVenta,ImporteCoste,CodTienda
0,14,S1-12-001-1014067,173210,03000264,U,1.0,1.5045,1.0465,001
1,15,S1-12-001-1014067,173211,04000383,U,1.0,0.8925,0.4255,001
2,21,S1-12-001-1018802,180371,02000443,U,2.0,2.1250,1.9090,001
3,22,S1-12-001-1018802,180372,02000442,U,1.0,1.5555,1.4950,001
4,28,S1-12-001-1024113,188001,03000287,U,1.0,1.7425,1.1270,001


In [80]:
df.dropna(axis=0,inplace=True)

In [81]:
df.groupby('CodTienda').count().reset_index().sort_values('Ticket', ascending = False).head()

,CodTienda,index,Ticket,Linea,CodProducto,UnidadMedida,Cantidad,ImporteVenta,ImporteCoste
2,005,128426,128426,128426,128426,128426,128426,128426,128426
5,011,97876,97876,97876,97876,97876,97876,97876,97876
0,001,56173,56173,56173,56173,56173,56173,56173,56173
4,009,38847,38847,38847,38847,38847,38847,38847,38847
3,006,17322,17322,17322,17322,17322,17322,17322,17322


In [82]:
df = (df[df['CodTienda']=="005"].groupby(['Ticket', 'CodProducto'])['Cantidad'].sum().unstack().reset_index().fillna(0).set_index('Ticket'))



In [83]:
df

CodProducto,01000002,01000003,01000004,01000005,01000014,01000015,01000016,01000017,01000022,01000023,...,13000117,13000128,13000167,13000168,13000169,13000170,13000171,13000172,13000174,13000175
Ticket,,,,,,,,,,,,,,,,,,,,,
S1-12-005-2280941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1-12-005-2281977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1-12-005-2283323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1-12-005-2287298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S1-12-005-2288103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S4-13-005-62778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4-13-005-62815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4-13-005-62873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
def sum_to_boolean(x):
    if x<=0:
        return 0
    else:
        return 1

X= df.applymap(sum_to_boolean)


In [85]:
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [87]:
frequent_itemsets = apriori(X, min_support = 0.006,use_colnames= True)
frequent_itemsets.sort_values('support', ascending = False).head()



,support,itemsets
77,0.170965,(04000383)
31,0.117699,(02001491)
75,0.059851,(04000380)
33,0.054177,(02001493)
58,0.048459,(04000055)


In [88]:
a_rules = association_rules(frequent_itemsets, metric = "lift", min_threshold= 1)
a_rules.sort_values('lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(02000655),(02000673),0.024118,0.022361,0.007587,0.314576,14.068261,0.007048,1.426327
3,(02000673),(02000655),0.022361,0.024118,0.007587,0.339303,14.068261,0.007048,1.477050
4,(02000655),(02001491),0.024118,0.117699,0.010657,0.441882,3.754326,0.007819,1.580849
5,(02001491),(02000655),0.117699,0.024118,0.010657,0.090548,3.754326,0.007819,1.073044
6,(02000673),(02001491),0.022361,0.117699,0.009523,0.425871,3.618290,0.006891,1.536763
7,(02001491),(02000673),0.117699,0.022361,0.009523,0.080907,3.618290,0.006891,1.063701
0,(02000654),(02001491),0.017021,0.117699,0.006452,0.379085,3.220789,0.004449,1.420968
1,(02001491),(02000654),0.117699,0.017021,0.006452,0.054820,3.220789,0.004449,1.039992
8,(04000383),(04000065),0.170965,0.030882,0.006586,0.038522,1.247373,0.001306,1.007945
9,(04000065),(04000383),0.030882,0.170965,0.006586,0.213256,1.247373,0.001306,1.053756
